In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


In [ ]:
loaded_keras_model = keras.models.load_model('../temp/outputs/09_01_tf_keras_save_model_and_weights/fashion_minist_model.h5')
loaded_keras_model(np.ones((1, 28, 28)))

In [ ]:
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
keras_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
keras_tflite = keras_to_tflite_converter.convert()

In [ ]:
if not os.path.exists('../temp/outputs/09_06_to_ftlite'):
    os.mkdir('../temp/outputs/09_06_to_ftlite')
with open('../temp/outputs/09_06_to_ftlite/keras_tflite_opt', 'wb') as f:
    f.write(keras_tflite)

In [ ]:
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype))

In [ ]:
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype = np.float32)))

In [ ]:
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_func])
concrete_func_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

In [ ]:
with open('../temp/outputs/09_06_to_ftlite/concrete_func_tflite_opt', 'wb') as f:
    f.write(keras_tflite)

In [ ]:
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('../temp/outputs/09_03_tf_keras_save_model/keras_saved_graph')
saved_model_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
saved_model_tflilte = saved_model_to_tflite_converter.convert()

In [ ]:
with open('../temp/outputs/09_06_to_ftlite/saved_model_tflite_opt', 'wb') as f:
    f.write(keras_tflite)